In [121]:
#!/usr/bin/python
#
# Brainfuck Interpreter
# Copyright 2011 Sebastian Kaspari
#
# Usage: ./brainfuck.py [FILE]

import sys

def execute(filename):
    f = open(filename, "r")
    evaluate(f.read())
    f.close()


def evaluate(code, inp=None, debug=False):
    code     = cleanup(list(code))
    bracemap = buildbracemap(code)

    cells, codeptr, cellptr = [0], 0, 0
    inputptr = 0

    try:
        while codeptr < len(code):
            command = code[codeptr]
            
            if debug:
                print(command, cellptr, cells)

            if command == ">":
                cellptr += 1
                if cellptr == len(cells): cells.append(0)

            if command == "<":
                cellptr = 0 if cellptr <= 0 else cellptr - 1

            if command == "+":
                cells[cellptr] = cells[cellptr] + 1 if cells[cellptr] < 255 else 0

            if command == "-":
                cells[cellptr] = cells[cellptr] - 1 if cells[cellptr] > 0 else 255

            if command == "[" and cells[cellptr] == 0: codeptr = bracemap[codeptr]
            if command == "]" and cells[cellptr] != 0: codeptr = bracemap[codeptr]
            if command == ".": sys.stdout.write(chr(cells[cellptr]))
            if command == ",": 
                if inp is not None:
                    if inputptr >= len(inp):
                        raise EOFError
                    cells[cellptr] = ord(inp[inputptr])
                    inputptr += 1
                else:
                    cells[cellptr] = ord(getch.getch())

            codeptr += 1
    except EOFError:
        pass
    return cells, codeptr, cellptr


def cleanup(code):
      return list(filter(lambda x: x in ['.', ',', '[', ']', '<', '>', '+', '-'], code))


def buildbracemap(code):
    temp_bracestack, bracemap = [], {}

    for position, command in enumerate(code):
        if command == "[": temp_bracestack.append(position)
        if command == "]":
            start = temp_bracestack.pop()
            bracemap[start] = position
            bracemap[position] = start
    return bracemap


# def main():
#     if len(sys.argv) == 2: execute(sys.argv[1])
#     else: print("Usage:", sys.argv[0], "filename")

# if __name__ == "__main__": main()


In [128]:
ord('v'), ord('^'), ord('=')

(118, 94, 61)

```
set cell 1 to 94
set cell 2 to 118-94=24
copy cell 1 into cell 0, using cell 3

set cell 5 to 0
read character into cell 6
while cell 6 != 0
  subtract cell 0 from cell 6
  if cell 6 == 0
     increment cell 5
  else
     copy cell 2 into cell 0, using cell 3
     subtract cell 0 from cell 6
     if cell 6 == 0
       decrement cell 5
  read character into cell 6
output cell 3
```


In [129]:
118-94

24

In [7]:
helloworld= '++++++++[>++++[>++>+++>+++>+<<<<-]>+>+>->>+[<]<-]>>.>---.+++++++..+++.>>.<-.<.+++.------.--------.>>+.>++.'

In [20]:
evaluate(helloworld)

Hello World!


([0, 0, 72, 100, 87, 33, 10], 106, 6)

In [19]:
evaluate(',[.,]', input='hello')

hello

([111], 3, 0)

In [196]:
program = '>' + '+' * 94 + '>' + '+' * 24
program += """
copy cell 1 into cell 0 using cell 3

<
[-<+>>>+<<]

>>
[-<<+>>]

read into cell 5
>>,

[
    subtract cell 0 from cell 5
    <<<<<[->>>>>-<<<<<]
    >>>>>

    if cell 5 != 0 do more
    [

      copy cell 2 into cell 0 using cell 3
      <<<[->+<<<+>>]

      move cell 3 into cell 2
      >[-<+>]
      <

      subtract cell 0 from cell 5
      <<[->>>>>-<<<<<]
      >>>>>

      if cell 5 != 0
      [
        increment cell 4 by 1
        <+>

        clear cell 5 to stop the loop
        [-]
      ]
      decrement cell 4 by 2
      <-->

    ]

    increment cell 4 by 1
    <+

    copy cell 1 into cell 0 using cell 3
    <<<[-<+>>>+<<]

    move cell 3 into cell 1
    >>[-<<+>>]

    >>

    read next input
    ,
]
write the output
<.

"""

In [197]:
evaluate(program, inp='^^v=^^^v')

([94, 94, 24, 0, 3, 0], 255, 5)

In [198]:
''.join(cleanup(program))

'>++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++>++++++++++++++++++++++++<[-<+>>>+<<]>>[-<<+>>]>>,[<<<<<[->>>>>-<<<<<]>>>>>[<<<[->+<<<+>>]>[-<+>]<<<[->>>>>-<<<<<]>>>>>[<+>[-]]<-->]<+<<<[-<+>>>+<<]>>[-<<+>>]>>,]<.'

In [123]:
inp = open('02-lifts.txt').read().strip()
evaluate(program, inp=inp)

([60, 60, 0, 15, 0], 152, 4)

In [124]:
def value(instr):
    if instr == '^':
        return 1
    elif instr == 'v':
        return -1
    else:
        return 0

In [125]:
def final(sequence):
    current = 0
    for c in sequence:
        current += value(c)
    return current

In [126]:
final(inp)

209

In [193]:
for i in range(50):
    print(final(inp[:i]), evaluate(program, inp[:i]), inp[:i])

0 ([94, 94, 24, 0, 0, 0], 144, 5) 
-1 ([94, 94, 24, 0, 255, 0], 255, 5) v
-2 ([94, 94, 24, 0, 254, 0], 255, 5) vv
-3 ([94, 94, 24, 0, 253, 0], 255, 5) vvv
-2 ([94, 94, 24, 0, 254, 0], 255, 5) vvv^
-1 ([94, 94, 24, 0, 255, 0], 255, 5) vvv^^
-2 ([94, 94, 24, 0, 254, 0], 255, 5) vvv^^v
-1 ([94, 94, 24, 0, 255, 0], 255, 5) vvv^^v^
-2 ([94, 94, 24, 0, 254, 0], 255, 5) vvv^^v^v
-2 ([94, 94, 24, 0, 254, 0], 255, 5) vvv^^v^v=
-3 ([94, 94, 24, 0, 253, 0], 255, 5) vvv^^v^v=v
-4 ([94, 94, 24, 0, 252, 0], 255, 5) vvv^^v^v=vv
-3 ([94, 94, 24, 0, 253, 0], 255, 5) vvv^^v^v=vv^
-4 ([94, 94, 24, 0, 252, 0], 255, 5) vvv^^v^v=vv^v
-3 ([94, 94, 24, 0, 253, 0], 255, 5) vvv^^v^v=vv^v^
-2 ([94, 94, 24, 0, 254, 0], 255, 5) vvv^^v^v=vv^v^^
-3 ([94, 94, 24, 0, 253, 0], 255, 5) vvv^^v^v=vv^v^^v
-4 ([94, 94, 24, 0, 252, 0], 255, 5) vvv^^v^v=vv^v^^vv
-5 ([94, 94, 24, 0, 251, 0], 255, 5) vvv^^v^v=vv^v^^vvv
-6 ([94, 94, 24, 0, 250, 0], 255, 5) vvv^^v^v=vv^v^^vvvv
-5 ([94, 94, 24, 0, 251, 0], 255, 5) vvv^^v^v=vv^v^^v

In [194]:
final(inp), evaluate(program, inp)

(209, ([94, 94, 24, 0, 209, 0], 255, 5))

In [199]:
! bf -n part1.bf < 02-lifts.txt > part1.bf.out